In [3]:
# naive deck generation

def gen_function(seed, num_run):
    rcards = ['1'] * 26 
    bcards = ['0'] * 26
    cards = rcards+bcards # total 52 cards
    total = []

    for gamenum in range(num_run):
        random.seed(seed)
        random.shuffle(cards)
        #print(cards)
        total.append(''.join(cards))
    return total

In [4]:
def game_function(deck, p1_seq, p2_seq):
    outcomes = []
    stack = [] # to keep track of the cards
    p1_num_tricks = 0
    p2_num_tricks = 0
    p1_num_cards = 0
    p2_num_cards = 0
    
    for card in deck:
        stack.append(card)
        curstack = ''.join(stack[-3:])
        if curstack == p1_seq:
            p1_num_tricks += 1
            p1_num_cards += len(stack)
            stack = []
        elif curstack == p2_seq:
            p2_num_tricks += 1
            p2_num_cards += len(stack)
            stack = []
    return p1_num_tricks, p2_num_tricks, p1_num_cards, p2_num_cards

In [5]:
# base pseudocode
def whole_simulation(seed, num_run):
    i = 0
    game_df = pd.DataFrame(columns = ['seed', 'deck', 'p1_seq', 'p2_seq', 'p1_num_tricks', 'p2_num_tricks', 'p1_num_cards', 'p2_num_cards'])
    decks = gen_function(seed, num_run)
    for deck in decks:
        for p1_seq in ['000', '001', '010', '011', '100', '101', '110', '111']:
            for p2_seq in ['000', '001', '010', '011', '100', '101', '110', '111']:
                p1_num_tricks, p2_num_tricks, p1_num_cards, p2_num_cards = game_function(deck, p1_seq, p2_seq)
                game_df.loc[i] = [seed, deck, p1_seq, p2_seq, p1_num_tricks, p2_num_tricks, p1_num_cards, p2_num_cards]
                i += 1
    return game_df

In [59]:
import random
import numpy as np
import sqlite3
import pandas as pd

In [60]:
import penney_db1
from importlib import reload
reload(penney_db1);
from  penney_db1 import DB

In [61]:
simulation = whole_simulation(1, 10)

In [62]:
simulation.head()

,seed,deck,p1_seq,p2_seq,p1_num_tricks,p2_num_tricks,p1_num_cards,p2_num_cards
0,1,0101101101000110010010011001011110100101110000...,000,000,2,0,45,0
1,1,0101101101000110010010011001011110100101110000...,000,001,2,6,20,32
2,1,0101101101000110010010011001011110100101110000...,000,010,1,7,6,39
3,1,0101101101000110010010011001011110100101110000...,000,011,2,6,9,39
4,1,0101101101000110010010011001011110100101110000...,000,100,0,8,0,51


In [63]:
def make_database(simulation):
    db = DB()
    db.connect_db()
    db.insert_results(simulation)
    return db.get_database_file()

In [64]:
database = make_database(simulation)

In [1]:
def probabilities(database, ):
    conn = sqlite3.connect(database)
    cursor = conn.cursor()
    
    sql1 ='''
    CREATE VIEW win_results AS
    SELECT 
        p1_seq,
        p2_seq,
        p1_num_cards,
        p2_num_cards,
        p1_num_tricks,
        p2_num_tricks,
        deck,
        CASE 
            WHEN p1_num_cards > p2_num_cards THEN 'p1'
            WHEN p1_num_cards < p2_num_cards THEN 'p2'
            ELSE 'draw'
    END AS win_cards,
    CASE 
        WHEN p1_num_tricks > p2_num_tricks THEN 'p1'
        WHEN p1_num_tricks < p2_num_tricks THEN 'p2'
        ELSE 'draw'
    END AS win_tricks
    FROM 
    results;
    '''
    db.run_action(sql1)
    sql2 = '''
    WITH TricksWins AS (
        SELECT 
            all_combinations.p1_seq, 
            all_combinations.p2_seq, 
            COALESCE(COUNT(win_results.win_tricks), 0) AS p1_wins_by_tricks
        FROM 
            (SELECT DISTINCT p1_seq, p2_seq FROM win_results) AS all_combinations
        LEFT JOIN 
            win_results ON all_combinations.p1_seq = win_results.p1_seq 
                        AND all_combinations.p2_seq = win_results.p2_seq
                        AND win_results.win_tricks = 'p1'
        GROUP BY 
            all_combinations.p1_seq, 
            all_combinations.p2_seq
    ), 
    CardWins AS (
        SELECT 
            all_combinations.p1_seq, 
            all_combinations.p2_seq, 
            COALESCE(COUNT(win_results.win_cards), 0) AS p1_wins_by_cards
        FROM 
            (SELECT DISTINCT p1_seq, p2_seq FROM win_results) AS all_combinations
        LEFT JOIN 
            win_results ON all_combinations.p1_seq = win_results.p1_seq 
                        AND all_combinations.p2_seq = win_results.p2_seq
                        AND win_results.win_cards = 'p1'
        GROUP BY 
            all_combinations.p1_seq, 
            all_combinations.p2_seq
    ),
    TimesPlayed AS (
        SELECT 
            p1_seq, 
            p2_seq, 
            COUNT(*) AS TimesPlayed
        FROM 
            win_results
        GROUP BY 
            p1_seq, 
            p2_seq
    )
    
    SELECT 
        t.p1_seq, 
        t.p2_seq, 
        IFNULL(t.p1_wins_by_tricks, 0) AS P1TrickWinCount,
        IFNULL(c.p1_wins_by_cards, 0) AS P1CardWinCount,
        tp.TimesPlayed, 
        IFNULL(1.0 * t.p1_wins_by_tricks / tp.TimesPlayed, 0) AS P1ProbWinTricks,
        IFNULL(1.0 * c.p1_wins_by_cards / tp.TimesPlayed, 0) AS P1ProbWinCard
    FROM 
        TricksWins t
    LEFT JOIN 
        CardWins c ON t.p1_seq = c.p1_seq AND t.p2_seq = c.p2_seq
    LEFT JOIN 
        TimesPlayed tp ON t.p1_seq = tp.p1_seq AND t.p2_seq = tp.p2_seq;
    '''
    prob_df = db.run_query(sql2)
    
    prob_p1card=prob_df['P1ProbWinCard'].values
    prob_p1trick=prob_df['P1ProbWinTrick'].values
    prob_p1card=np.reshape(prob_df['P1ProbWinCard'].values, (8,8))
    prob_p1trick=np.reshape(prob_df['P1ProbWinTrick'].values, (8,8))

    return prob_p1card, prob_p1trick